In [1]:
from lxml import etree
import pandas as pd
from io import StringIO
import requests
from urllib.parse import quote_plus

In [2]:
# url = 'https://www.fpds.gov/ezsearch/fpdsportal?s=FPDS&indexName=awardfull&templateName=1.5.2&q=LAST_MOD_DATE%3A%5B2020%2F04%2F02%2C2020%2F04%2F02%5D&rss=1&feed=atom0.3'
# url = 'https://www.fpds.gov/ezsearch/fpdsportal?s=FPDS.GOV&indexName=icdindex&templateName=1.5.2&q=+DEPARTMENT_NAME%3A%22General+Services+Administration%22&rss=1&feed=atom0.3'
# url = 'https://www.fpds.gov/ezsearch/search.do?indexName=awardfull&templateName=1.5.2&s=FPDS.GOV&q=financial+assistance+actions&rss=1&feed=atom0.3'
# url = 'https://www.fpds.gov/ezsearch/fpdsportal?s=FPDS.GOV&indexName=awardfull&templateName=1.5.2&rss=1&feed=atom0.3&q=+LAST_MOD_DATE%3A%5B2021%2F10%2F01%2C2021%2F11%2F01%5D'

base_url = 'https://www.fpds.gov/ezsearch/fpdsportal?s=FPDS.GOV&indexName=awardfull&templateName=1.5.2&rss=1&feed=atom0.3&q=+'
start_date = '2020/04/02'
end_date = '2020/04/02'
param = f'LAST_MOD_DATE:[{start_date},{end_date}]'

url = base_url + quote_plus(param)

print(url)


https://www.fpds.gov/ezsearch/fpdsportal?s=FPDS.GOV&indexName=awardfull&templateName=1.5.2&rss=1&feed=atom0.3&q=+LAST_MOD_DATE%3A%5B2020%2F04%2F02%2C2020%2F04%2F02%5D


In [ ]:
response = requests.get(url)
print(response.ok)

In [ ]:
doc = etree.XML(response.content)

In [ ]:
xslt_tree = etree.parse("fpds_feed.xsl")
transform = etree.XSLT(xslt_tree)

In [ ]:
result = transform(doc)

In [ ]:
csv_str =str(result)
# csv_str

In [ ]:
mystr = StringIO(csv_str)

df = pd.read_csv(mystr, sep="\t", header=None, index_col=0)
df.head(10)

In [ ]:
df_new = df.transpose(copy=True)
df_new.iloc[:, :10]

In [ ]:
df_new.shape

In [ ]:
# output dataframe to a .csv file
df_new.to_csv("fpds_feed_result.csv", index=False)

# Get count of records returned

In [ ]:
f = StringIO(response.text)
tree = etree.XML(response.content)
href = tree.xpath("/f:feed/f:link[@rel = 'last']/@href", namespaces={'f': 'http://www.w3.org/2005/Atom'})[0]
print(href)

In [ ]:
import re

x = re.search("start=(\d+)$", href)
print(x.group(1))


# Test fpds_feed_reader

In [ ]:
from fpds_feed import fpds_feed_reader

In [ ]:
reader = fpds_feed_reader()


In [ ]:
df = reader.get_data("2021/11/01", "2021/11/15")
df.head(10)

# Test Multi-threaded Requests 

In [ ]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from time import time

url_list = [(start, url + f"&start={start}") for start in range(0, 100, 10)]
# print(url_list)

def download_file(url, index):
    response = requests.get(url, stream=True)
    # return response.content
    return index

start = time()

processes = []
with ThreadPoolExecutor(max_workers=10) as executor:
    for url in url_list:
        processes.append(executor.submit(download_file, url[1], url[0]))

for task in as_completed(processes):
    print(task.result())


print(f'Time taken: {time() - start}')

In [3]:
from multithreaded_requests import requests as t_requests
from data_transforms import transforms

In [7]:
req = t_requests(base_url)
iter = req.get(param, start_index=0, max_workers=1, page_size=10)
# for task in iter:
#     print(len(task.result()))
print(next(iter).result())

b'<?xml version="1.0" encoding="UTF-8"?>\n<feed xmlns="http://www.w3.org/2005/Atom">\n  <title>FPDS-NG search results for<![CDATA[:  LAST_MOD_DATE:[2020/04/02,2020/04/02]]]></title>\n  <link rel="alternate" type="text/html" href="https://www.fpds.gov/ezsearch/search.do?s=FPDS&amp;indexName=awardfull&amp;templateName=1.5.2&amp;q=+LAST_MOD_DATE%3A%5B2020%2F04%2F02%2C2020%2F04%2F02%5D&amp;start=0"></link>\n  <link rel="last" type="text/html" href="https://www.fpds.gov/ezsearch/FEEDS/ATOM?s=FPDS&amp;FEEDNAME=PUBLIC&amp;VERSION=1.5.2&amp;q=+LAST_MOD_DATE%3A%5B2020%2F04%2F02%2C2020%2F04%2F02%5D&amp;start=33000"></link>\n  <link rel="next" type="text/html" href="https://www.fpds.gov/ezsearch/FEEDS/ATOM?s=FPDS&amp;FEEDNAME=PUBLIC&amp;VERSION=1.5.2&amp;q=+LAST_MOD_DATE%3A%5B2020%2F04%2F02%2C2020%2F04%2F02%5D&amp;start=10"></link>\n  <modified/>\n  <author>\n    <name/>\n  </author>\n  <entry>\n    <title><![CDATA[New DELIVERY ORDER 0001 awarded to SARAS AMERICA INC. for the amount of $2,501]]><